<a href="https://colab.research.google.com/github/kouliki/Sentiment_Analysis-using-Textblob/blob/main/Sentiment_analysis_using_Textblob_GN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from textblob import TextBlob
from nltk.tokenize.toktok import ToktokTokenizer
import re
tokenizer = ToktokTokenizer()
import spacy
nlp = spacy.load('en_core_web_sm',disable=['ner'])

In [4]:
TextBlob("I am happy").sentiment

# polarity range from[-1,1] and subjectivity ranges [0,1]

Sentiment(polarity=0.8, subjectivity=1.0)

In [3]:
TextBlob("he is not a good boy").sentiment

Sentiment(polarity=-0.35, subjectivity=0.6000000000000001)

### Data Loading
dataset used : https://www.kaggle.com/datasets/columbine/imdb-dataset-sentiment-analysis-in-csv-format

In [8]:
! pip install -q kaggle

In [9]:
from google.colab import files
files.upload()

Saving kaggle (6).json to kaggle (6).json


{'kaggle (6).json': b'{"username":"koulikimahato","key":"b6ce180c38dd23a9a62574b9da0bc963"}'}

In [11]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [12]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [13]:
 ! kaggle datasets list

ref                                                                 title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rabieelkharoua/students-performance-dataset                         📚 Students Performance Dataset 📚                     66KB  2024-06-12 23:09:20          13275        273  1.0              
nelgiriyewithana/most-streamed-spotify-songs-2024                   Most Streamed Spotify Songs 2024                    496KB  2024-06-15 18:50:51          10075        210  1.0              
ambaliyagati/spotify-dataset-for-playing-around-with-sql            Spotify dataset                                     302KB  2024-06-17 10:48:20           1975         43  1.0              
ihelon/coffee-sales                     

In [14]:
! kaggle datasets download -d columbine/imdb-dataset-sentiment-analysis-in-csv-format

Dataset URL: https://www.kaggle.com/datasets/columbine/imdb-dataset-sentiment-analysis-in-csv-format
License(s): world-bank
 35% 9.00M/25.7M [00:00<00:00, 21.4MB/s]
100% 25.7M/25.7M [00:00<00:00, 56.0MB/s]


In [15]:
! unzip imdb-dataset-sentiment-analysis-in-csv-format

Archive:  imdb-dataset-sentiment-analysis-in-csv-format.zip
  inflating: Test.csv                
  inflating: Train.csv               
  inflating: Valid.csv               


In [16]:
train=pd.read_csv("Train.csv")
# test=pd.read_csv("test.csv")

In [17]:
label_0=train[train['label']==0].sample(n=5000)
label_1=train[train['label']==1].sample(n=5000)
train=pd.concat([label_0,label_1])
from sklearn.utils import shuffle
train=shuffle(train)
# train=train.sample(frac=1).reset_index(drop=True

In [18]:
train

,text,label
33375,I have always wanted to see this because I lov...,0
5915,"I saw ""Paris Je T'Aime"" because a friend reall...",0
29210,Don't be taken in because the premise of this ...,0
11073,The film opens with Bill Coles (Melvyn Douglas...,1
21609,I recently had the idea to make a short film f...,0
...,...,...
32468,"""You know - I've been in some towns where the ...",0
15520,The guy mentioned to sue for the 1.5 hours was...,0
33335,When I first watched this movie I thought it w...,1
17331,My reasoning behind viewing this film (despite...,0


### Data PreProcessing

In [19]:
train.isnull().sum()

text     0
label    0
dtype: int64

In [20]:
import numpy as np
train.replace(r'^\s"$', np.nan,regex=True, inplace=True)
print('escape seq removed')

escape seq removed


In [21]:
train

,text,label
33375,I have always wanted to see this because I lov...,0
5915,"I saw ""Paris Je T'Aime"" because a friend reall...",0
29210,Don't be taken in because the premise of this ...,0
11073,The film opens with Bill Coles (Melvyn Douglas...,1
21609,I recently had the idea to make a short film f...,0
...,...,...
32468,"""You know - I've been in some towns where the ...",0
15520,The guy mentioned to sue for the 1.5 hours was...,0
33335,When I first watched this movie I thought it w...,1
17331,My reasoning behind viewing this film (despite...,0


In [22]:
# remove the non-ascii words

train['text']=train['text'].str.encode('ascii','ignore').str.decode('ascii')
print('non-ascii data removed')

non-ascii data removed


In [23]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [24]:
# remove punctuation
def remove_punctuation(text):
  import string
  for punctuation in string.punctuation:
    text=text.replace(punctuation,'')
  return text
train['text']=train['text'].apply(remove_punctuation)
print('punctuation removed')

punctuation removed


In [25]:
train

,text,label
33375,I have always wanted to see this because I lov...,0
5915,I saw Paris Je TAime because a friend really w...,0
29210,Dont be taken in because the premise of this f...,0
11073,The film opens with Bill Coles Melvyn Douglas ...,1
21609,I recently had the idea to make a short film f...,0
...,...,...
32468,You know Ive been in some towns where the gir...,0
15520,The guy mentioned to sue for the 15 hours wast...,0
33335,When I first watched this movie I thought it w...,1
17331,My reasoning behind viewing this film despite ...,0


In [27]:
# stopwords

## removing stopwords


import nltk
nltk.download('stopwords')

import nltk
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [28]:
import nltk
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [31]:
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove('no') ## not and no are removed
stopword_list.remove('not')

In [30]:
def custom_remove_stopwords(text,is_lower_case=False):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  if is_lower_case:
    filtered_tokens=[token for token in tokens if token not in stopword_list]
  else:
    filtered_tokens=[token for token in tokens if token.lower() not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text


In [32]:
train['text']=train['text'].apply(custom_remove_stopwords)
print('stopwords removed')

stopwords removed


In [33]:
train

,text,label
33375,always wanted see love cheesy horror movies ti...,0
5915,saw Paris Je TAime friend really wanted see we...,0
29210,Dont taken premise film good one not good film...,0
11073,film opens Bill Coles Melvyn Douglas telling s...,1
21609,recently idea make short film featuring man hi...,0
...,...,...
32468,know Ive towns girls werent pretty fact Ive to...,0
15520,guy mentioned sue 15 hours wasted life cant di...,0
33335,first watched movie thought strange movie know...,1
17331,reasoning behind viewing film despite fact fre...,0


In [34]:
## removing special characters

def remove_special_characters(text):
  pattern=r'[^a-zA-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text

In [35]:
train['text']=train['text'].apply(remove_special_characters)
print('special characters removed')

special characters removed


In [37]:
def remove_html(text):
  import re ## Regular expressions are powerful tools for pattern matching in text
  html=re.compile(r'<.*?>')
  return html.sub(r'',text)

In [38]:
train['text']=train['text'].apply(remove_html)
print('html tags removed')

html tags removed


In [39]:
def remove_URL(text):
  url=re.compile(r'https?://\S+|www\.\S+')
  return url.sub(r'',text)

In [40]:
train['text']=train['text'].apply(remove_URL)
print('URL removed')

URL removed


In [41]:
def remove_numbers(text):
  result=re.sub(r'\d+','',text)
  return result

In [42]:
train['text']=train['text'].apply(remove_numbers)
print('numbers removed')

numbers removed


In [43]:
def cleanse(word):
  rx=re.compile(r'\D"\d')
  if rx.match(word):
    return ''
  return word
def remove_alphanumeric(strings):
  nstrings=[" ".join(filter(None,(
      cleanse(word) for word in string.split())))
  for string in strings.split()]
  str1=' '.join(nstrings)
  return str1



In [44]:
train['text']=train['text'].apply(remove_alphanumeric)
print('alphanumeric removed')

alphanumeric removed


In [45]:
train

,text,label
33375,always wanted see love cheesy horror movies ti...,0
5915,saw Paris Je TAime friend really wanted see we...,0
29210,Dont taken premise film good one not good film...,0
11073,film opens Bill Coles Melvyn Douglas telling s...,1
21609,recently idea make short film featuring man hi...,0
...,...,...
32468,know Ive towns girls werent pretty fact Ive to...,0
15520,guy mentioned sue hours wasted life cant disag...,0
33335,first watched movie thought strange movie know...,1
17331,reasoning behind viewing film despite fact fre...,0


### Lemmatization

In [46]:
def lemmatize_text(text):
  text=nlp(text)
  text=' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
  return text

In [47]:
train['text']=train['text'].apply(lemmatize_text)
print('lemmatization done')

lemmatization done


In [48]:
train['sentiment']=train['text'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [49]:
sentiment_series=train['sentiment'].tolist()

In [53]:
# Calculate both polarity and subjectivity
train['polarity'] = train['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
train['subjectivity'] = train['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

# Create a list of tuples containing both values
sentiment_series = train[['polarity', 'subjectivity']].values.tolist()

# Now create the DataFrame
columns = ['polarity', 'subjectivity']
df1 = pd.DataFrame(sentiment_series, columns=columns, index=train.index)

In [54]:
result=pd.concat([train,df1],axis=1)

In [55]:
result.drop('sentiment',axis=1,inplace=True)

In [60]:
# Calculate both polarity and subjectivity
train['polarity'] = train['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
train['subjectivity'] = train['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

# Create a list of tuples containing both values
# Convert the series to a list of dictionaries to preserve column names
sentiment_series = train[['polarity', 'subjectivity']].to_dict(orient='records')

# Now create the DataFrame
df1 = pd.DataFrame(sentiment_series, index=train.index)

result = pd.concat([train, df1], axis=1)
result.drop('sentiment', axis=1, inplace=True)

# # result.loc[result['polarity'] >= 0.3, 'Sentiment'] = 'Positive'  # Create 'Sentiment' column
# result.loc[result['polarity'] < 0, 'label'] = 'Neutral'
# result.loc[result['polarity'] <= -0.3, 'Sentiment'] = 'Negative'

In [61]:
result

,text,label,polarity,subjectivity,polarity,subjectivity
33375,always want see love cheesy horror movie title...,0,0.082502,0.568106,0.082502,0.568106
5915,see Paris Je TAime friend really want see go a...,0,-0.014675,0.427427,-0.014675,0.427427
29210,do not take premise film good one not good fil...,0,-0.010565,0.522321,-0.010565,0.522321
11073,film open Bill Coles Melvyn Douglas tell story...,1,0.132742,0.535784,0.132742,0.535784
21609,recently idea make short film feature man hit ...,0,-0.114917,0.496036,-0.114917,0.496036
...,...,...,...,...,...,...
32468,know I ve town girl be not pretty fact I ve to...,0,0.048513,0.590751,0.048513,0.590751
15520,guy mention sue hour waste life can not disagr...,0,-0.027323,0.579186,-0.027323,0.579186
33335,first watch movie think strange movie know dir...,1,0.014087,0.307123,0.014087,0.307123
17331,reason behind view film despite fact free less...,0,-0.021280,0.523115,-0.021280,0.523115


In [62]:
result.loc[result['label']==1,'sentiment']=1
result.loc[result['label']==0,'sentiment']=0

In [63]:
result

,text,label,polarity,subjectivity,polarity,subjectivity,sentiment
33375,always want see love cheesy horror movie title...,0,0.082502,0.568106,0.082502,0.568106,0.0
5915,see Paris Je TAime friend really want see go a...,0,-0.014675,0.427427,-0.014675,0.427427,0.0
29210,do not take premise film good one not good fil...,0,-0.010565,0.522321,-0.010565,0.522321,0.0
11073,film open Bill Coles Melvyn Douglas tell story...,1,0.132742,0.535784,0.132742,0.535784,1.0
21609,recently idea make short film feature man hit ...,0,-0.114917,0.496036,-0.114917,0.496036,0.0
...,...,...,...,...,...,...,...
32468,know I ve town girl be not pretty fact I ve to...,0,0.048513,0.590751,0.048513,0.590751,0.0
15520,guy mention sue hour waste life can not disagr...,0,-0.027323,0.579186,-0.027323,0.579186,0.0
33335,first watch movie think strange movie know dir...,1,0.014087,0.307123,0.014087,0.307123,1.0
17331,reason behind view film despite fact free less...,0,-0.021280,0.523115,-0.021280,0.523115,0.0
